In [ ]:
import calendar
import logging
import pickle
import re
from pathlib import Path

import numpy as np
import pandas as pd
import pypsa
import xarray as xr
from _helpers import configure_logging, get_efficiency

logger = logging.getLogger(__name__)

from _helpers import calculate_annual_investment

if __name__ == "__main__":

    scenario = snakemake.params["scenario"]
    configure_logging(snakemake)

    supply = xr.open_dataset(snakemake.input["supply"])

    # Load additional components
    with open(snakemake.input["additional_components"], "rb") as f:
        override_component_attrs = pickle.load(f)

    network = pypsa.Network(override_component_attrs=override_component_attrs)
    network.import_from_netcdf(snakemake.input["network"])

    Path(snakemake.output["lcoes"]).parent.mkdir(parents=True, exist_ok=True)

    # Load annual demand (consider overwrite option in config)
    if scenario["synthetic_demand"].lower() in ["gegis", "custom"]:
        demand = pd.read_csv(snakemake.input["demand"], sep=";", index_col="region")

        demand = demand.loc[snakemake.wildcards["from"], "demand [GWh]"]
        demand *= 1e3  # in MWh

    else:
        logging.error(
            f"Option {scenario['synthetic_demand']} in config for scenario -> synthethic_demand unknown."
        )

    # Apply scenario modifier
    demand *= scenario["modifiers"]["synthetic_demand"]
    logger.info(f"Removing {demand} MWh of annual demand from potential supply.")

    capacities = supply["capacities"]

    capacity_factors = supply["profiles"].mean(dim="time")

    # Determine which generation capacities are available for export
    # and which are needed for satisfying domestic electricity demand
    # Simple heuristic:
    # 1.) Consider annual demand and generation per source+source class
    # 2.) Reserve (Exclude) sources with lowest LCoE for demand
    # 3.) Source + source class where demand and generation overlap has
    #     its generation capacitiy partially reduced based on share of overlap.

    # 1.) name mapping between dataset and cost.csv file for technologies
    # TODO no longer needed without GEGIS, happens in combine_suppply
    mapping = {
        "offwind": "offwind",
        "onwind": "onwind",
        "solar-utility": "solar-utility",
        "csp-tower": "solar-utility",  # TODO add CSP to costs.csv and change here
    }

    wacc = pd.read_csv(snakemake.input["wacc"], comment="#", index_col="region")
    wacc = wacc.loc[snakemake.wildcards["from"], scenario["wacc"]]
    wacc *= scenario["modifiers"]["wacc"]  # Apply scenario modifier

    def calculate_investment(name):
        # Calculate annual investment for technology "x"

        if name == "csp-tower":  # TODO add CSP to costs.csv and then remove here
            name = "solar-utility"  # TODO add CSP to costs.csv and then remove here

        invest = calculate_annual_investment(name, wacc, snakemake.input["costs"])
        invest *= 1.0e3  # Investment costs in costs.data in kW, here MW

        return invest

    ## Annual investment per maximum capacity of each class and technology
    # Investment per MW capacity (use pandas mapping, doesn't seem to work on xarray)
    investment = (
        capacities["technology"].to_pandas().map(calculate_investment).to_xarray()
    )
    investment = investment * capacities
    investment = investment.rename("annual investment")

    # Calculate annual generation
    generation = (supply["profiles"] * capacities).fillna(0.0).sum(dim="time")
    generation = generation.rename("annual generation")

    # Calculate LCoE
    lcoe = investment / generation
    lcoe = lcoe.rename("lcoe")

    # Convert capacities and LCoE into an easier to handle format
    df = xr.merge(
        [
            capacities,
            lcoe,
            generation,
        ]
    ).to_dataframe()

    df = df.dropna().sort_values("lcoe")

    # Calculate annual generation across all sources + source classes
    df["cumulative generation"] = df.cumsum()["annual generation"]

    # Remove inf values (espc. for LCoE where generation = 0)
    df = df.replace([np.inf, -np.inf], np.nan).dropna()

    # Save LCoEs as intermediary result
    df.to_csv(snakemake.output["lcoes"])

    ## Remaining demand overlapping into higher source class

    fully_reserved_capacity = df[df["cumulative generation"] <= demand]
    if fully_reserved_capacity.empty:  # regions with lowest class outproducing demand
        fully_reserved_capacity = 0
    else:  # regions with some classes fully reserved for domestic demand
        fully_reserved_capacity = fully_reserved_capacity.iloc[-1][
            "cumulative generation"
        ]

    residual_demand = demand - fully_reserved_capacity

    # 2.) Exclude sources reserved for domestic demand
    df = df[df["cumulative generation"] > demand]

    if df.empty:
        logger.error(
            f"Insufficient renewable capacities ({generation.sum().item():.0f} MWh)"
            f" to satisfy domestic demand ({demand:.0f} MWh)."
        )

    # 3.) Reduce capacitiy at overlap
    residual_demand_share = residual_demand / df.iloc[0]["annual generation"]

    df.iloc[0]["capacities"] *= 1.0 - residual_demand_share

    df = df.reorder_levels(["technology", "class"]).sort_index()
    for idx, row in df.iterrows():

        # Select profiles as maximum dispatchable feed-in
        # Lower and upper clipping
        # max 100% (higher values due to EPS)
        # min 0.1% or 0. (easier for solving and non-relevant generation)
        p_max_pu = supply["profiles"].sel({"technology":idx[0], "class": idx[1]}, drop=True)
        p_max_pu = p_max_pu.clip(min=0., max=1.0)
        p_max_pu = p_max_pu.where(lambda x: x > 0.0001, 0).to_numpy()


        if idx[0] in {"solar-utility","offwind","onwind"}:
            # These technologies directly feed-in electricity
            network.add(
                "Generator",
                name=f"{idx[0]} {idx[1]}",
                bus="electricity (exp)",
                p_nom_max=row["capacities"],
                p_nom_extendable=True,
                p_max_pu=p_max_pu,
                # Cost data for kW, here MW implicitly assumed
                capital_cost=1.0e3
                * calculate_annual_investment(idx[0], wacc, snakemake.input["costs"]),
                carrier=idx[0],
            )
        elif idx[0] == "csp-tower":
            # CSP is modelled with the generator as heat provider ("csp-tower #")
            # and additional components for heat storage ("csp-tower TES #"),
            # heat-to-electricity generator ("csp-tower power block #")
            # and a dedicated bus for each CSP plant class ("csp-tower bus #")
            bus_name = f"{idx[0]} {idx[1]} bus"
            network.add(
                "Bus",
                name=bus_name,
                carrier="heat",
                unit="MW",
            )

            name_ = f"{idx[0]}"
            network.add(
                "Generator",
                name=f"{name_} {idx[1]}",
                bus=bus_name,
                p_nom_max=row["capacities"],
                p_nom_extendable=True,
                p_max_pu=p_max_pu,
                # Cost data for kW, here MW implicitly assumed
                capital_cost=1.0e3
                * calculate_annual_investment(name_, wacc, snakemake.input["costs"]),
                carrier=f"{idx[0]}",
            )

            name_ = f"{idx[0]} TES"
            network.add(
                "Store",
                name=f"{name_} {idx[1]}",
                bus=bus_name,
                e_nom_extendable=True,
                e_cyclic=True,
                capital_cost=calculate_annual_investment(name_, wacc, snakemake.input["costs"]),
            )

            name_ = f"{idx[0]} power block"
            efficiency_ = get_efficiency(name_, snakemake.input["efficiencies"])
            network.add(
                "Link",
                name=f"{name_} {idx[1]}",
                bus0=bus_name,
                bus1="electricity (exp)",
                p_nom_extendable=True,
                efficiency=efficiency_,
                # capital_cost are scaled with efficiency as capital cost are given in MW_e in cost data
                capital_cost=efficiency_ * calculate_annual_investment(name_, wacc, snakemake.input["costs"]),
            )

        else:
            logger.error(f"Unknown technology '{idx[0]}'.")

network.export_to_netcdf(snakemake.output["network"])

supply.close()